In [4]:
import numpy as np
from collections import defaultdict

def leiden_algorithm(adj_matrix, resolution_parameter):
    # Convert adjacency matrix to networkx graph
    import networkx as nx
    G = nx.from_numpy_array(adj_matrix)

    # Initialize partition, node-to-community map, and modularity
    partition = list(range(len(G.nodes)))
    node2com = {node: i for i, node in enumerate(G.nodes)}
    modularity = -np.inf

    # Iterate until no further improvement in modularity
    while True:
        improved = False

        # Randomize order of nodes
        nodes = list(G.nodes)
        np.random.shuffle(nodes)

        # Iterate over nodes in randomized order
        for node in nodes:
            # Get current community of node
            cur_com = node2com[node]

            # Compute the modularity gain of moving node to each neighboring community
            neighbor_coms = defaultdict(int)
            for neighbor in G.neighbors(node):
                neighbor_com = node2com[neighbor]
                if neighbor_com != cur_com:
                    neighbor_coms[neighbor_com] += adj_matrix[node, neighbor] - resolution_parameter * np.sum(adj_matrix[node, :]) * np.sum(adj_matrix[:, neighbor]) / np.sum(adj_matrix)

            # Get community with maximum modularity gain
            if neighbor_coms:
                best_com = max(neighbor_coms, key=neighbor_coms.get)
                gain = neighbor_coms[best_com]
            else:
                gain = 0

            # Move node to best community if modularity is improved
            if gain > 0:
                partition[node] = best_com
                node2com[node] = best_com
                modularity += gain
                improved = True

        # Break if no further improvement in modularity
        if not improved:
            break

    # Return partition and modularity
    return partition, modularity


In [5]:
# Generate random adjacency matrix
n = 50  # number of nodes
p = 0.1  # edge probability
adj_matrix = np.random.rand(n, n) < p
adj_matrix = np.triu(adj_matrix) + np.triu(adj_matrix, 1).T

In [6]:
leiden_algorithm(adj_matrix, 0.5)